In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 9


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-07 10:45:24] ax.service.ax_client: Generated new trial 144 with parameters {'Drug': 87, 'SL_1': 64, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 1, 'S_3': 0, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the re

8 min


In [6]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,1,0,0
1,0,0,4,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,1,0,0,0


In [7]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
2           146    88    64     0     0     0     0     0    0    0  ...    0   
3           147    88    64     0     0     0     0     0    0    0  ...    0   
4           148    92    72     0     0     0     0     0    0    0  ...    0   
5           149    92    72     0     0     0     0     0    0    0  ...    0   
6           150    77    79     0     0     0     0     0    0    0  ...    0   
7           151    88    64     0     0     0     0     0    0    0  ...    0   
8           152    88    64     0     0     0     0     0    0    0  ...    0   
10          154    88    64     0     0     0     0     0    0    0  ...    0   
12          156    88    64     0     0     0     0     0    0    0  ...    0   
14          158    88    64     0     0     0     0     0    0    0  ...    0   
15          159    88    64     0     0     0     0     0    0    0  ...    0   

In [8]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,1,0,0
1,0,0,4,0
2,0,0,0,100
3,0,0,0,100
4,0,0,0,100
5,0,0,0,100
6,0,0,0,100
7,0,0,0,100
8,0,0,0,100
9,1,0,0,0


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0000,16.000000,16.000000
mean,151.500000,0.564563,0.435438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.0625,0.125000,0.687500
std,4.760952,0.024822,0.024822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.341565,0.2500,0.341565,0.478714
min,144.000000,0.494000,0.421000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.000000
25%,147.750000,0.560250,0.421000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.000000
50%,151.500000,0.578000,0.422000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,1.000000
75%,155.250000,0.579000,0.439750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,1.000000
max,159.000000,0.579000,0.506000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.0000,1.000000,1.000000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0000,16.000000,16.000000
mean,151.500000,89.312500,69.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187500,0.0625,0.312500,68.750000
std,4.760952,4.908751,7.898312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.543906,0.2500,1.014479,47.871355
min,144.000000,77.000000,64.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.000000
25%,147.750000,88.000000,64.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.000000
50%,151.500000,88.000000,64.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,100.000000
75%,155.250000,91.250000,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,100.000000
max,159.000000,100.000000,93.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,1.0000,4.000000,100.000000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.000000,16.000000,16.0
mean,151.500000,197.596875,152.403125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.000000,62.5,125.000000,687.500000,0.0
std,4.760952,8.687682,8.687682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341.565026,250.0,341.565026,478.713554,0.0
min,144.000000,172.900000,147.350000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
25%,147.750000,196.087500,147.350000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
50%,151.500000,202.300000,147.700000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1000.000000,0.0
75%,155.250000,202.650000,153.912500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1000.000000,0.0
max,159.000000,202.650000,177.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.000000,1000.0,1000.000000,1000.000000,0.0


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        3.2
SL_1        2.4
SL_2        0.0
SL_3        0.0
LL_1        0.0
LL_2        0.0
LL_3        0.0
P_1         0.0
P_2         0.0
P_3         0.0
S_1         2.0
S_2         1.0
S_3         2.0
Water      11.0
Solvent     0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_9_OT_2_protocol_1.py',
 'protocol/iteration_9_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,144,0.576,0.424,0,0,0,0,0,0,0,0,0,1,0,0,2,0
1,145,0.556,0.444,0,0,0,0,0,0,0,0,0,0,1,0,2,0
2,146,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,147,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,148,0.561,0.439,0,0,0,0,0,0,0,0,0,0,0,1,1,0
5,149,0.561,0.439,0,0,0,0,0,0,0,0,0,0,0,1,1,0
6,150,0.494,0.506,0,0,0,0,0,0,0,0,0,0,0,1,1,0
7,151,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
8,152,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
9,153,0.558,0.442,0,0,0,0,0,0,0,0,1,0,0,0,2,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Caution,A1,181.485175,0.570796
1,Caution,A2,275.509173,0.570796
2,Caution,A3,129.827966,0.570796
3,Good,A4,230.514802,0.263062
4,Good,A5,227.314699,0.209770
5,Good,A6,231.573529,0.194689
6,Caution,B1,256.180808,0.176482
7,Caution,B2,266.947565,0.399785
8,Caution,B3,259.720175,0.416023
9,Good,B4,253.298503,0.256319


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,229.801010,2.217326,0.222507,0.035922,1
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,248.043106,7.432253,0.246533,0.013839,1
4,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,Formulation 9,254.045374,10.603279,0.245521,0.088767,1
9,Formulation 10,278.327418,11.487048,0.261255,0.087921,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,144,0.576,0.424,0,0,0,0,0,0,0,...,0,0,2,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,145,0.556,0.444,0,0,0,0,0,0,0,...,1,0,2,0,Formulation 2,229.801010,2.217326,0.222507,0.035922,1
2,146,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,147,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 4,248.043106,7.432253,0.246533,0.013839,1
4,148,0.561,0.439,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,149,0.561,0.439,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,150,0.494,0.506,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,151,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,152,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 9,254.045374,10.603279,0.245521,0.088767,1
9,153,0.558,0.442,0,0,0,0,0,0,0,...,0,0,2,0,Formulation 10,278.327418,11.487048,0.261255,0.087921,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,144,0.576,0.424,0,0,0,0,0,0,0,...,2,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,145,0.556,0.444,0,0,0,0,0,0,0,...,2,0,Formulation 2,229.801010,2.217326,0.222507,0.035922,1,1112.0,0
2,146,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,147,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 4,248.043106,7.432253,0.246533,0.013839,1,1158.0,0
4,148,0.561,0.439,0,0,0,0,0,0,0,...,1,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
5,149,0.561,0.439,0,0,0,0,0,0,0,...,1,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,150,0.494,0.506,0,0,0,0,0,0,0,...,1,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,151,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
8,152,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 9,254.045374,10.603279,0.245521,0.088767,1,1158.0,0
9,153,0.558,0.442,0,0,0,0,0,0,0,...,2,0,Formulation 10,278.327418,11.487048,0.261255,0.087921,1,1116.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
1,1,0.229801,0.002217,0.556,0.0,0.222507,0.035922,0.166667
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
3,1,0.248043,0.007432,0.579,0.0,0.246533,0.013839,0.083333
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
8,1,0.254045,0.010603,0.579,0.0,0.245521,0.088767,0.083333
9,1,0.278327,0.011487,0.558,0.0,0.261255,0.087921,0.166667


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
1,1,0.229801,0.002217,0.556,0.0,0.222507,0.035922,0.166667
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
3,1,0.248043,0.007432,0.579,0.0,0.246533,0.013839,0.083333
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
8,1,0.254045,0.010603,0.579,0.0,0.245521,0.088767,0.083333
9,1,0.278327,0.011487,0.558,0.0,0.261255,0.087921,0.166667


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,144,0.576,0.424,0,0,0,0,0,0,0,...,0.166667,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.000,0
1,145,0.556,0.444,0,0,0,0,0,0,0,...,0.166667,0,Formulation 2,0.229801,0.002217,0.222507,0.035922,1,0.556,0
2,146,0.579,0.421,0,0,0,0,0,0,0,...,0.083333,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0.000,0
3,147,0.579,0.421,0,0,0,0,0,0,0,...,0.083333,0,Formulation 4,0.248043,0.007432,0.246533,0.013839,1,0.579,0
4,148,0.561,0.439,0,0,0,0,0,0,0,...,0.083333,0,Formulation 5,1.000000,0.000000,1.000000,0.000000,0,0.000,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-07 13:49:23] ax.service.ax_client: Completed trial 144 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 13:49:23] ax.service.ax_client: Completed trial 145 with data: {'Solu': (0.556, 0), 'Size': (0.229801, 0.002217), 'PDI': (0.222507, 0.035922)}.
[INFO 08-07 13:49:23] ax.service.ax_client: Completed trial 146 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 13:49:23] ax.service.ax_client: Completed trial 147 with data: {'Solu': (0.579, 0), 'Size': (0.248043, 0.007432), 'PDI': (0.246533, 0.013839)}.
[INFO 08-07 13:49:23] ax.service.ax_client: Completed trial 148 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 13:49:23] ax.service.ax_client: Completed trial 149 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 13:49:23] ax.service.ax_client: Completed trial 150 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 13:49: